In [1]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.svm import SVC
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [8]:
data = pd.read_json('disambiguated_dataTrain.json')
data.drop(axis=1, columns='expandedText', inplace=True)
display(data)
data.columns = ['id', 'label','text']
stop_words= stopwords.words('english')
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf = vectorizer.fit_transform(data['text'])
test_data = pd.read_json('dataTest.json')

,id,label,text
0,aaexyuw,course,571 main page_n_03 computer_n_01 science_n_01 ...
1,abbdqt,course,object_n_01 oriented programming alumnus_n_01 ...
2,achmly,course,ece carbon_n_01 752 spring_n_04 1996 ece vitam...
3,aciio,course,last modified thursday 01 february_n_01 96 22 ...
4,ackfxrep,course,eecs401 web_n_01 page_n_01 fall_n_05 96 welcom...
...,...,...,...
6618,zxgxje,student,home_page_n_01 daqing lithium_n_01 welcome wor...
6619,zyidaxg,student,last modified sunday 25 aug 96 22 35 51 gmt pe...
6620,zyrphu,student,home marla baker_n_01 marla carbon_n_01 capita...
6621,zyvupbc,student,patrice caire patrice caire virtual environmen...


In [9]:
X_train = data['text']
y_train = data['label']
cfl = SVC()
params = {
    'C': [0.1, 1, 2]
}

model = gs_knn = GridSearchCV(cfl,
                      param_grid=params,
                      scoring='f1_weighted',
                      cv=10)
model.fit(tfidf, y_train)
best_params = model.best_params_
best_model = model.best_estimator_
print(best_params)

{'C': 2}


In [10]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, data['label'], test_size=0.2, random_state=42)
model = SVC(class_weight='balanced', C=2,random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

report = classification_report(y_test, y_pred, labels=data['label'].unique())
print(report)

In [8]:
model = SVC(class_weight='balanced',random_state=42)
model.fit(tfidf, data['label'])
test_tfidf = vectorizer.transform(test_data['text'])
y_pred = model.predict(test_tfidf)

In [10]:
result_df = pd.DataFrame({'id': test_data.id, 'Predicted': y_pred})
result_df

,id,Predicted
0,aaclkul,other
1,aagelci,other
2,aangjmn,other
3,aawnpc,other
4,abdjgiz,student
...,...,...
1654,zxmmn,other
1655,zxwkru,other
1656,zybimtt,student
1657,zypnixf,faculty


In [11]:
result_df.to_csv('ENXEBRE_SVM_weightbalanced_without_extend.csv')